In [9]:
from sshtunnel import SSHTunnelForwarder
import paramiko
import pandas as pd
import neo4j as neo
from neo4j import GraphDatabase
import psycopg2 as pg
from datetime import datetime, timedelta
import psycopg2
import pandas as pd
from oauth2client.service_account import ServiceAccountCredentials
from sshtunnel import SSHTunnelForwarder
from dbutils.pooled_db import PooledDB

In [10]:
# Neo4j 연결 함수 정의
def create_neo4j_connection(uri, username="neo4j", password="s8713753!@"):
    driver = GraphDatabase.driver('bolt://localhost:7687', auth=(username, password), max_connection_lifetime=3600*24*30, keep_alive=True)
    return driver

# Neo4j 연결 생성
driver = create_neo4j_connection("bolt://localhost:7687")

# 연결 테스트
def test_connection(driver):
    with driver.session() as session:
        result = session.run("MATCH (n) RETURN count(n) as count")
        count = result.single()["count"]
        print(f"데이터베이스의 총 노드 수: {count}")

# 연결 테스트 실행
test_connection(driver)

# 사용이 끝난 후에는 반드시 연결을 닫아주세요
# neo4j_driver.close()

데이터베이스의 총 노드 수: 643


In [11]:
ssh_host = 'api.trustyapp.io'
ssh_username = 'ubuntu'
ssh_pem_file = 'ng-prod.pem'
database_host = '127.0.0.1'  # SSH 터널을 통해 localhost로 접근
database_name = 'trusty'
database_username = 'server'
database_password = 'ygFAhsyTMDzB21VdLTnoq7djxwjDNzuc9yPGukm73otXER8ihD'

server = SSHTunnelForwarder(
    (ssh_host, 22),  # SSH 서버 주소와 포트
    ssh_username=ssh_username,
    ssh_pkey=ssh_pem_file,  # Private key 파일 경로
    remote_bind_address=(database_host, 5432)  # 원격 MySQL 서버 주소와 포트
)

server.start()

conn = psycopg2.connect(
    host='127.0.0.1',  # localhost를 통해 SSH 터널로 접속
    user=database_username,
    password=database_password,
    database=database_name,
    port=server.local_bind_port  # sshtunnel이 자동으로 할당한 포트
)

In [12]:
qry = '''
SELECT  "id", "uuid", "username", "createdAt", "firstName", "middleName", "lastName", "birthDate", "bvnCertStatus", "csChannelVerifiedAt", "earningCertStatus", "cbCheckedAt", "creditLimit", "remainingLimit", "usedFirstPromoAt", "membershipLevel", "helperFriendChosenAt", "helperId", "installHelperId", "contactsAppPrivacyAgreedAt", "helpedUserCount"
FROM "user";
'''

df = pd.read_sql(qry, conn)
df

,id,uuid,username,createdAt,firstName,middleName,lastName,birthDate,bvnCertStatus,csChannelVerifiedAt,...,cbCheckedAt,creditLimit,remainingLimit,usedFirstPromoAt,membershipLevel,helperFriendChosenAt,helperId,installHelperId,contactsAppPrivacyAgreedAt,helpedUserCount
0,6,69a9cc07-d491-48d0-a51f-a5805571ea5b,Ayfaymenah,2024-10-24 11:52:10.045794+00:00,Efe,None,Umolo,1990-06-04,approved,2024-11-14 05:48:53.732000+00:00,...,NaT,0,0,NaT,None,NaT,NaN,NaN,NaT,0
1,8,4ebe1170-c8c4-4d29-b680-5aa2eef544d8,Angelopeters1,2024-11-04 13:31:44.078665+00:00,Timothy,Emeka,Eze,1997-09-15,approved,2024-11-04 13:34:54.688000+00:00,...,NaT,5000,5000,NaT,bronze,2024-11-05 05:10:41.878000+00:00,4.0,NaN,NaT,0
2,10,52783d88-fa0a-40f3-85c0-350965a6f80f,Gold,2024-11-06 08:31:08.047980+00:00,Gbenga,None,Gbadebo,None,initialized,NaT,...,NaT,0,0,NaT,None,NaT,NaN,NaN,NaT,0
3,3,0c4235dd-437b-4913-b90b-e74761b639fe,Jonathan,2024-10-24 11:14:44.027384+00:00,Jonathan,Olusola,Omoluabi,1997-12-06,approved,2024-11-13 05:04:56.442000+00:00,...,2024-11-13 10:54:20.859000+00:00,5000,5000,NaT,bronze,2024-11-13 05:11:14.024000+00:00,NaN,NaN,NaT,0
4,27,ceb0731e-1aaa-4969-80ea-4cb53cef8eb5,olawore,2024-11-14 16:30:23.470196+00:00,Ojo,Akabi,Olawore,1995-05-05,approved,2024-11-26 01:09:33.110000+00:00,...,NaT,0,0,NaT,None,NaT,NaN,NaN,NaT,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
730,736,3079c4cd-4ffb-4fc2-873e-1f0d2e633af2,Ademor22,2025-01-30 15:59:28.854997+00:00,Olumide,Ademola,Odesanya,None,initialized,NaT,...,NaT,0,0,NaT,None,NaT,NaN,NaN,NaT,0
731,728,ec909410-a8c6-4f04-b97e-06ca7b79d411,Philips33,2025-01-29 15:45:43.773604+00:00,Ugochukwu,Philip,Okereke,1998-10-15,approved,2025-01-29 15:48:33.127000+00:00,...,NaT,0,0,NaT,None,NaT,NaN,NaN,2025-01-29 15:52:47.920000+00:00,0
732,732,93947b83-1b90-417a-9e10-57d7fad81a2e,Dotun,2025-01-30 08:12:46.865036+00:00,Adedotun,Joseph,Adebowale,1965-08-29,approved,2025-01-30 08:14:02.005000+00:00,...,NaT,0,0,NaT,None,NaT,NaN,NaN,NaT,0
733,733,9a55fa3e-683b-4cdc-a750-8d30e4c8e5c4,ibrahimshitta@yahoo.com,2025-01-30 12:36:47.921830+00:00,Ibrahim,Adeolu,Shitta,None,initialized,NaT,...,NaT,0,0,NaT,None,NaT,NaN,NaN,NaT,0


In [13]:
users = df.to_dict(orient='records')

# Timestamp를 문자열로 변환
for user in users:
    user['createdAt'] = user['createdAt'].isoformat() if pd.notnull(user['createdAt']) else None
    user['csChannelVerifiedAt'] = user['csChannelVerifiedAt'].isoformat() if pd.notnull(user['csChannelVerifiedAt']) else None
    user['cbCheckedAt'] = user['cbCheckedAt'].isoformat() if pd.notnull(user['cbCheckedAt']) else None
    user['usedFirstPromoAt'] = user['usedFirstPromoAt'].isoformat() if pd.notnull(user['usedFirstPromoAt']) else None
    user['birthDate'] = user['birthDate'].isoformat() if pd.notnull(user['birthDate']) else None
    user['contactsAppPrivacyAgreedAt'] = user['contactsAppPrivacyAgreedAt'].isoformat() if pd.notnull(user['contactsAppPrivacyAgreedAt']) else None
    user['helperFriendChosenAt'] = user['helperFriendChosenAt'].isoformat() if pd.notnull(user['helperFriendChosenAt']) else None

qry = '''
    UNWIND $users AS user
    MERGE (u:User {id: user.id})
    SET u.uuid = user.uuid,
        u.username = user.username,
        u.createdAt = user.createdAt,
        u.firstName = user.firstName,
        u.middleName = user.middleName,
        u.lastName = user.lastName,
        u.birthDate = user.birthDate,
        u.bvnCertStatus = user.bvnCertStatus,
        u.csChannelVerifiedAt = user.csChannelVerifiedAt,
        u.earningCertStatus = user.earningCertStatus,
        u.cbCheckedAt = user.cbCheckedAt,
        u.creditLimit = user.creditLimit,
        u.remainingLimit = user.remainingLimit,
        u.usedFirstPromoAt = user.usedFirstPromoAt,
        u.membershipLevel = user.membershipLevel,
        u.contactsAppPrivacyAgreedAt = user.contactsAppPrivacyAgreedAt,
        u.helperFriendChosenAt = user.helperFriendChosenAt;
'''

with driver.session() as session:
    session.run(qry, users=users)

In [14]:
qry = '''
    UNWIND $users AS user
    MATCH (u:User {id: user.id})
    OPTIONAL MATCH (hu: User {id: user.helperId})
    WITH u, hu WHERE hu IS NOT NULL
    MERGE (u)-[:HELPED]->(hu);
'''

with driver.session() as session:
    session.run(qry, users=users)

In [15]:
qry = '''
    UNWIND $users AS user
    MATCH (u:User {id: user.id})
    OPTIONAL MATCH (ihu: User {id: user.installHelperId})
    WITH u, ihu WHERE ihu IS NOT NULL
    MERGE (u)-[:INSTALL_HELPED]->(ihu);
'''

with driver.session() as session:
    session.run(qry, users=users)

In [16]:
qry = '''
    SELECT "friendship"."userId", "friend"."userId"
    FROM "friendship"
    INNER JOIN "personal_identifier" "friend" ON "friendship"."friendId" = "friend"."userId"
    WHERE "friend"."userId" IS NOT NULL
'''

with conn.cursor() as cursor:
    cursor.execute(qry)
    friendships = cursor.fetchall()

friendships

[(1, 440),
 (1, 442),
 (1, 456),
 (1, 457),
 (1, 468),
 (1, 472),
 (1, 474),
 (1, 477),
 (1, 478),
 (1, 483),
 (1, 485),
 (1, 519),
 (1, 537),
 (1, 538),
 (1, 556),
 (1, 564),
 (1, 585),
 (1, 586),
 (1, 598),
 (1, 602),
 (1, 613),
 (1, 620),
 (1, 621),
 (1, 637),
 (1, 644),
 (1, 651),
 (1, 652),
 (1, 696),
 (1, 718),
 (1, 726),
 (1, 3),
 (1, 16),
 (1, 21),
 (1, 47),
 (1, 50),
 (1, 52),
 (1, 60),
 (1, 66),
 (1, 112),
 (1, 114),
 (1, 116),
 (1, 126),
 (1, 173),
 (1, 175),
 (1, 146),
 (1, 155),
 (1, 156),
 (1, 158),
 (1, 160),
 (1, 162),
 (1, 163),
 (1, 168),
 (1, 199),
 (1, 200),
 (1, 257),
 (1, 262),
 (1, 235),
 (1, 238),
 (1, 239),
 (1, 240),
 (1, 241),
 (1, 244),
 (1, 245),
 (1, 508),
 (1, 322),
 (1, 324),
 (1, 356),
 (1, 359),
 (1, 369),
 (1, 370),
 (1, 373),
 (1, 374),
 (1, 377),
 (1, 387),
 (1, 390),
 (1, 392),
 (1, 394),
 (1, 410),
 (1, 443),
 (1, 446),
 (1, 448),
 (1, 452),
 (1, 488),
 (1, 505),
 (1, 516),
 (1, 522),
 (1, 523),
 (1, 524),
 (1, 714),
 (1, 543),
 (1, 547),
 (1, 551

In [17]:
qry = '''
    MATCH (u:User {id: $friendship.userId})
    MATCH (f:User {id: $friendship.friendId})
    MERGE (u)-[:HAS_CONTACT_OF]->(f);
'''

with driver.session() as session:
    for f in friendships:
        friendship = {
            'userId': f[0],
            'friendId': f[1]
        }
        session.run(qry, friendship=friendship)



In [18]:
qry = '''
    SELECT "userId", "guaranteeTargetId", "evaluationResult"
    FROM "guarantee"
    WHERE "evaluationResult" IS NOT NULL
'''

with conn.cursor() as cursor:
    cursor.execute(qry)
    guarantees = cursor.fetchall()

guarantees

[(613, 613, 'later'),
 (282, 416, 'later'),
 (75, 185, 'trust'),
 (75, 5, 'trust'),
 (63, 321, 'trust'),
 (63, 384, 'trust'),
 (63, 435, 'trust'),
 (30, 307, 'trust'),
 (127, 7, 'trust'),
 (489, 505, 'trust'),
 (30, 98, 'trust'),
 (418, 515, 'trust'),
 (134, 525, 'trust'),
 (109, 127, 'trust'),
 (109, 7, 'trust'),
 (134, 529, 'trust'),
 (612, 636, 'trust'),
 (488, 539, 'trust'),
 (668, 714, 'trust'),
 (488, 527, 'trust'),
 (534, 488, 'trust'),
 (534, 534, 'trust'),
 (668, 607, 'trust'),
 (11, 5, 'trust'),
 (11, 9, 'trust'),
 (11, 243, 'trust'),
 (527, 528, 'later'),
 (418, 643, 'trust'),
 (418, 645, 'trust'),
 (418, 637, 'trust'),
 (515, 643, 'trust'),
 (515, 572, 'trust'),
 (612, 646, 'trust'),
 (668, 628, 'trust'),
 (668, 636, 'trust'),
 (439, 656, 'later'),
 (527, 488, 'trust'),
 (127, 109, 'trust'),
 (127, 162, 'trust'),
 (515, 418, 'trust'),
 (515, 515, 'trust'),
 (506, 543, 'trust'),
 (7, 163, 'trust'),
 (7, 127, 'trust'),
 (7, 103, 'trust'),
 (7, 109, 'trust'),
 (418, 558, 'trus

In [19]:
for g in guarantees:
    relation_name = 'TRUSTED' if g[2] == 'trust' else 'LATERED'
    qry = f'''
        MATCH (u:User {{id: $guarantee.userId}})
        MATCH (gt:User {{id: $guarantee.guaranteeTargetId}})
        MERGE (u)-[:{relation_name}]->(gt);
    '''
    with driver.session() as session:
        guarantee = {
            'userId': g[0],
            'guaranteeTargetId': g[1],
        }
        session.run(qry, guarantee=guarantee)



In [20]:
salary_users = [4,
5,
8,
11,
13,
20,
23,
30,
38,
75,
87,
98,
120,
125,
180,
216,
217,
245,
270,
282,
295,
295,
313,
410,
488,
506,
508,
515,
522,
527,
533,
534,
539,
546,
552,
562,
563,
573,
582,
583,
584,
598,
600,
602,
608,
612,
622,
635,
637]

In [21]:
for _id in salary_users:
    qry = f'''
    MATCH (u:User)
    WHERE u.id = {_id}
    SET u:salary_user
    '''

    with driver.session() as session:
        session.run(qry, users=users)


In [22]:
qry = '''
    MATCH (u:User)
    WHERE u.usedFirstPromoAt IS NOT NULL
    SET u:SilverUser
'''

with driver.session() as session:
    session.run(qry, users=users)

In [23]:
from datetime import datetime, timedelta
import psycopg2
import pandas as pd
import gspread
from oauth2client.service_account import ServiceAccountCredentials
from sshtunnel import SSHTunnelForwarder
from dbutils.pooled_db import PooledDB

ssh_host = 'api.trustyapp.io'
ssh_username = 'ubuntu'
ssh_pem_file = 'ng-prod.pem'
database_host = '127.0.0.1'  # SSH 터널을 통해 localhost로 접근
database_name = 'trusty'
database_username = 'server'
database_password = 'ygFAhsyTMDzB21VdLTnoq7djxwjDNzuc9yPGukm73otXER8ihD'

server = SSHTunnelForwarder(
    (ssh_host, 22),  # SSH 서버 주소와 포트
    ssh_username=ssh_username,
    ssh_pkey=ssh_pem_file,  # Private key 파일 경로
    remote_bind_address=(database_host, 5432)  # 원격 MySQL 서버 주소와 포트
)

server.start()

conn = psycopg2.connect(
    host='127.0.0.1',  # localhost를 통해 SSH 터널로 접속
    user=database_username,
    password=database_password,
    database=database_name,
    port=server.local_bind_port  # sshtunnel이 자동으로 할당한 포트
)
cursor = conn.cursor()

qry = '''
    select "user_test"."targetUserId"
    from "user_test"
'''

cursor.execute(qry)
results = cursor.fetchall()

df = pd.DataFrame(results)
df.columns = ['test_id']  # 컬럼 이름 변경
test_id_list = list(df['test_id'])

# 쿼리 다 쓰면 ssh 터널 닫기
conn.close()
server.stop()

In [24]:
for _id in test_id_list:
    qry = f'''
    MATCH (u:User)
    WHERE u.id = {_id}
    SET u:test_user
    '''

    with driver.session() as session:
        session.run(qry, users=users)

In [25]:
conn.close()